In [200]:
import os
import sys
import importlib

import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import redfin

importlib.reload(redfin)

pd.set_option('display.max_columns', None)

In [39]:
!pwd
!ls


/Users/timcodes/ds/metis/metisgh/02_Luther/Luther
2013_movies.csv                geckodriver.log
LICENSE                        pickles
README.md                      rappold_tim_challenge_3.ipynb
__pycache__                    redfin.py
data.txt                       redfin_2018-04-18-08-51-54.csv
explore_luther.ipynb


['.DS_Store',
 'listing_urls_10024page_18.pkl',
 'listing_urls_11211page_18.pkl',
 'listing_urls_48503page_18.pkl',
 'listing_urls_77373page_18.pkl',
 'listing_urls_90049page_18.pkl',
 'listing_urls_90292page_18.pkl',
 'listing_urls_90301page_18.pkl',
 'listing_urls_90403page_18.pkl',
 'listing_urls_94110page_18.pkl',
 'listing_urls_94549page_18.pkl',
 'listing_urls_94605page_18.pkl',
 'listing_urls_94610page_18.pkl',
 'listing_urls_94611page_18.pkl',
 'listing_urls_95476page_18.pkl']

In [181]:
list_of_all_urls = redfin.load_everything_pickle()

In [182]:
len(list_of_all_urls)

4893

In [203]:
np.random.choice(small_list, size=7, replace=False, p=None)

array(['/CA/Sonoma/431-Chase-St-95476/home/2259714',
       '/CA/Lafayette/1008-Woodbury-Rd-94549/unit-101/home/112888202',
       '/NY/New-York/186-W-80th-St-10024/unit-7K/home/45273346',
       '/TX/Spring/5015-Fitzwater-Dr-77373/home/30314611',
       '/CA/Sonoma/790-Peru-Rd-95476/home/2497327',
       '/CA/Lafayette/3164-Cordova-Way-94549/home/997787',
       '/NY/New-York/331-W-87th-St-10024/home/45275791'], dtype='<U68')

In [205]:
pickle_file='pickles/home_stats_all.pkl'
with open(pickle_file, 'rb') as picklefile:
    dummydata = pickle.load(picklefile)

In [212]:
len(dummydata)

169

In [213]:
dummydata

[{'APN': '022180008',
  'Baths': '—',
  'Beds': '—',
  'County': 'Kings County',
  'Finished Sq. Ft.': '3,360',
  'Last Sold': 'Aug 2, 2017',
  'Lot Size': '1,627 Sq. Ft.',
  'Sales Price': '$1,550,000',
  'Stories': '3',
  'Style': 'Multi-Family (2-4 Unit)',
  'Total Sq. Ft.': '3,360',
  'Unfinished Sq. Ft.': '—',
  'Year Built': '1899',
  'Year Renovated': '1899',
  'Zip Code': '11211',
  'rURL': '/NY/Unknown/645-Bedford-Ave-11211/home/40736142'},
 {'APN': '0983910000234',
  'Baths': '2.5',
  'Beds': '3',
  'County': 'Harris County',
  'Finished Sq. Ft.': '2,844',
  'Last Sold': None,
  'Lot Size': '0.43 Acres',
  'Sales Price': None,
  'Stories': '2',
  'Style': 'Single Family Residential',
  'Total Sq. Ft.': '2,844',
  'Unfinished Sq. Ft.': '—',
  'Year Built': '1988',
  'Year Renovated': '1988',
  'Zip Code': '77373',
  'rURL': '/TX/Spring/650-Ames-St-77373/home/30205884'},
 {'APN': '4024031032',
  'Baths': '1',
  'Beds': '2',
  'County': 'Los Angeles County',
  'Finished Sq. Ft.'

In [209]:
test_list = redfin.load_everything_pickle()
test_list = np.random.choice(test_list, size=30, replace=False)

In [211]:
len(test_list)

30

In [201]:
list_of_dicts = redfin.scrape_home_stats(small_list) 

[1.09635192]
[0.85611547]
[0.91849305]
[1.13450089]
[1.27882127]
[0.87275112]
[1.0392008]
[0.99001226]
[0.92645475]
[0.75134836]
[0.96558815]
[0.82800422]
[0.9958491]
[0.91788269]
[1.23145852]
[1.21240069]
[1.02667102]
[0.96670493]
[0.98789296]
[1.09727557]
[0.84082517]
[1.06050691]
[0.93518316]
[1.32983183]
[0.77878658]
[0.93448865]
[0.83747099]
[1.24118213]
[0.92364671]
[1.03562889]


In [202]:
pd.DataFrame(list_of_dicts)

,APN,Baths,Beds,County,Finished Sq. Ft.,Last Sold,Lot Size,Sales Price,Stories,Style,Total Sq. Ft.,Unfinished Sq. Ft.,Year Built,Year Renovated,Zip Code,rURL
0,011084101900,3,4,Alameda County,"2,841","Dec 24, 2015","3,390 Sq. Ft.","$931,000",3,Multi-Family (2-4 Unit),"2,841",—,1948,1948,94610,/CA/Oakland/888-Warfield-Ave-94610/home/1881044
1,023671019,—,—,Kings County,—,"Aug 31, 2016",—,"$893,468",6,Condo/Co-op,—,—,—,—,11211,/NY/Unknown/280-Metropolitan-Ave-11211/unit-4F...
2,018381029,2,3,Sonoma County,"1,347","Apr 6, 2018","5,985 Sq. Ft.","$975,000",1,Single Family Residential,"1,347",—,1956,1985,95476,/CA/Sonoma/431-Chase-St-95476/home/2259714
3,4281032039,2,3,Los Angeles County,"1,141","Aug 30, 2016",—,"$910,000",—,Condo/Co-op,"1,141",—,1962,1963,90403,/CA/Santa-Monica/837-Lincoln-Blvd-90403/unit-2...
4,4013401001,4,4,Genesee County,"2,464","Feb 20, 2018","6,098 Sq. Ft.","$150,000",—,Single Family Residential,"3,168",704,1890,—,48503,/MI/Flint/503-Hazelton-St-48503/home/111442706
5,910003256,—,—,Sonoma County,—,None,—,None,—,Mobile/Manufactured Home,—,—,—,—,95476,/CA/Sonoma/189-Argentina-Flag-Way-95476/home/4...
6,1202100020017,2,3,Harris County,"1,777",None,"6,050 Sq. Ft.",None,1,Single Family Residential,"1,777",—,2003,2003,77373,/TX/Spring/23119-Seminole-Park-Ln-77373/home/3...
7,126031028,2,2,Sonoma County,"1,377","Apr 23, 2015",5.85 Acres,"$1,000,000",2,Single Family Residential,"1,377",—,1910,1960,95476,/CA/Sonoma/790-Peru-Rd-95476/home/2497327
8,4224018022,2,2,Los Angeles County,"1,582","Jun 14, 2016",1.64 Acres,"$600,000",—,Townhouse,"1,582",—,1968,1968,90292,/CA/Marina-del-Rey/4808-La-Villa-Marina-90292/...
9,043457001100,2,3,Alameda County,"1,427","Jun 22, 2015","4,000 Sq. Ft.","$346,000",1,Single Family Residential,"1,427",—,1914,1965,94605,/CA/Oakland/2255-86th-Ave-94605/home/1712655


In [197]:
d = [{'points': 50, 'time': '5:00', 'year': 2010}, 
{'points': 25, 'time': '6:00', 'month': "february"}, 
{'points':90, 'time': '9:00', 'month': 'january'}, 
{'points_h1':20, 'month': 'june'}]

In [198]:
pd.DataFrame(d)

,month,points,points_h1,time,year
0,NaN,50.0,NaN,5:00,2010.0
1,february,25.0,NaN,6:00,NaN
2,january,90.0,NaN,9:00,NaN
3,june,NaN,20.0,NaN,NaN


In [173]:
columns = ['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY', 'STATE', 'ZIP',
           'PRICE', 'BEDS', 'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
           'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
           'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED',
           'LATITUDE', 'LONGITUDE']

In [174]:
pd.Series(index=columns)

SALE TYPE                    NaN
SOLD DATE                    NaN
PROPERTY TYPE                NaN
ADDRESS                      NaN
CITY                         NaN
STATE                        NaN
ZIP                          NaN
PRICE                        NaN
BEDS                         NaN
BATHS                        NaN
LOCATION                     NaN
SQUARE FEET                  NaN
LOT SIZE                     NaN
YEAR BUILT                   NaN
DAYS ON MARKET               NaN
$/SQUARE FEET                NaN
HOA/MONTH                    NaN
STATUS                       NaN
NEXT OPEN HOUSE START TIME   NaN
NEXT OPEN HOUSE END TIME     NaN
URL                          NaN
SOURCE                       NaN
MLS#                         NaN
FAVORITE                     NaN
INTERESTED                   NaN
LATITUDE                     NaN
LONGITUDE                    NaN
dtype: float64

In [ ]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

## Figure out Filtered Landing Page
- Verify the problem using requests and bs4: get home_listing_url and verify that its status is `active` or `new` and not `sold`
- Recreate get request using chrome driver and selenium:
Assuming you are on the page you want to parse, Selenium stores the source HTML in the driver's page_source attribute. You would then load the page_source into BeautifulSoup as follows:

```
In [8]: from bs4 import BeautifulSoup

In [9]: from selenium import webdriver

In [10]: driver = webdriver.Firefox()

In [11]: driver.get('http://news.ycombinator.com')

In [12]: html = driver.page_source

In [13]: soup = BeautifulSoup(html)

In [14]: for tag in soup.find_all('title'):
   ....:     print tag.text
   ....:  
```


In [232]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
import sys

chromedriver = "~/Downloads/chromedriver" # path to the chromedriver executable
chromedriver = os.path.expanduser(chromedriver)
print('chromedriver path: {}'.format(chromedriver))
sys.path.append(chromedriver)

driver = webdriver.Chrome(chromedriver)


chromedriver path: /Users/timcodes/Downloads/chromedriver


In [235]:
chrome_url = construct_redfin_url('94605', filter_='sold-3yr')
chrome_url

'https://www.redfin.com/zipcode/94605/filter/include=sold-3yr'

In [239]:
driver.get(chrome_url)
html = driver.page_source
selenium_soup = BeautifulSoup(html,"lxml")

selenium_list = find_home_listing_urls(selenium_soup)

In [240]:
selenium_list

['/CA/Oakland/2812-76th-Ave-94605/home/2036652',
 '/CA/Oakland/6129-Old-Quarry-Loop-94605/home/12115918',
 '/CA/Oakland/5921-Hilton-St-94605/home/568204',
 '/CA/Oakland/3055-Bantry-Ave-94605/home/1161573',
 '/CA/Oakland/4009-Malcolm-Ave-94605/home/1993133',
 '/CA/Oakland/8301-Aster-Ave-94605/home/1417868',
 '/CA/Oakland/8009-Earl-St-94605/home/579005',
 '/CA/Oakland/5528-Picardy-Dr-S-94605/home/567257',
 '/CA/Oakland/5514-Picardy-Dr-N-94605/home/1563765',
 '/CA/Oakland/7841-Surrey-Ln-94605/home/592451',
 '/CA/Oakland/4751-Dunkirk-Ave-94605/home/661538',
 '/CA/Oakland/3101-Partridge-Ave-94605/home/2040553',
 '/CA/Oakland/6144-Old-Quarry-Loop-94605/home/21670339',
 '/CA/Oakland/7560-Hansom-Dr-94605/home/843353',
 '/CA/Oakland/3240-62nd-Ave-94605/home/1925170',
 '/CA/Oakland/3680-Malcolm-Ave-94605/home/1930765',
 '/CA/Oakland/3200-Revere-Ave-94605/home/599097',
 '/CA/Oakland/60-Maggiora-Dr-94605/home/981030',
 '/CA/Oakland/2-Riviera-Ct-94605/home/1952844',
 '/CA/Oakland/5115-Fairhill-Ct-9

### Open downloaded CSV from Redfin to check its contents and use as a reference

In [37]:
!ls

LICENSE                        explore_luther.ipynb
README.md                      redfin_2018-04-18-08-51-54.csv


In [119]:
rf_csv = pd.read_csv('redfin_2018-04-18-08-51-54.csv')
rf_csv.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY', 'STATE',
       'ZIP', 'PRICE', 'BEDS', 'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS',
       'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [174]:
rf_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 27 columns):
SALE TYPE                                                                                     350 non-null object
SOLD DATE                                                                                     294 non-null object
PROPERTY TYPE                                                                                 350 non-null object
ADDRESS                                                                                       343 non-null object
CITY                                                                                          350 non-null object
STATE                                                                                         350 non-null object
ZIP                                                                                           350 non-null object
PRICE                                                                                         350

In [178]:
rf_csv.head()
list(rf_csv.columns)

['SALE TYPE',
 'SOLD DATE',
 'PROPERTY TYPE',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'PRICE',
 'BEDS',
 'BATHS',
 'LOCATION',
 'SQUARE FEET',
 'LOT SIZE',
 'YEAR BUILT',
 'DAYS ON MARKET',
 '$/SQUARE FEET',
 'HOA/MONTH',
 'STATUS',
 'NEXT OPEN HOUSE START TIME',
 'NEXT OPEN HOUSE END TIME',
 'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
 'SOURCE',
 'MLS#',
 'FAVORITE',
 'INTERESTED',
 'LATITUDE',
 'LONGITUDE']

In [184]:
rf_csv.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE,ZIP,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,April-4-2018,Single Family Residential,5514 Picardy Dr,Oakland,CA,94605,800000,4.0,2.0,Normandy Gardens,1650.0,3230.0,1926.0,14.0,485.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/5514-Picardy-...,EBRD/CCAR/Bay East,40815087,N,Y,37.775837,-122.190547
1,PAST SALE,April-16-2018,Multi-Family (2-4 Unit),2812 76th Ave,Oakland,CA,94605,555000,4.0,3.0,Eastmont Area,2576.0,4068.0,1940.0,2.0,215.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/2812-76th-Ave...,EBRD/CCAR/Bay East,40808499,N,Y,37.767668,-122.169261
2,PAST SALE,September-5-2017,Single Family Residential,4009 Malcolm Ave,Oakland,CA,94605,740000,3.0,2.5,Chabot Park,1948.0,9086.0,1960.0,225.0,380.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/4009-Malcolm-...,EBRD/CCAR/Bay East,40787391,N,Y,37.750808,-122.134996
3,PAST SALE,July-1-2017,Single Family Residential,8301 Aster Ave,Oakland,CA,94605,760000,4.0,2.0,Oakland,2354.0,6962.0,1938.0,291.0,323.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/8301-Aster-Av...,EBRD/CCAR/Bay East,40781777,N,Y,37.764025,-122.160524
4,PAST SALE,May-10-2017,Condo/Co-op,6144 Old Quarry Loop,Oakland,CA,94605,505000,2.0,2.0,East Oakl Hills,1079.0,NaN,2007.0,343.0,468.0,282.0,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/6144-Old-Quar...,EBRD/CCAR/Bay East,40775089,N,Y,37.778864,-122.165009
